# Chapter 9 - Structure from Motion

Structure from Motion (SFM) encompasses the application when one wants to get the 3D positions of the points in a setting where the input data is not a sequence of images (frames) but rather temporal independent images from different viewangles of the object. Usually in SFM it is possible that the images are takes by different cameras.

The overal problem formulation is like the following: Given a certain amount (n) point correspondences between two images we qant to estimate the 3D position, the camera pose and motion as well as the camera intrinsics.

If we work with calibrated cameras we obviously do not have to find the intrinsics K as they are already known.

## Scale Ambiguity

The first challenge we encounter in SFM is that since we do not have information about the real size and dimensions of the scene we can not find the correct scale. Rescaling the entire scene by a constant factor does not change the images and therefore stays undetected.

![Scale Ambiguity](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_9/1_scale_ambiguity.png)
*Figure 1: Scale Ambiguity. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*

As a result in monocular vision it is impossible to recover the absoculte scale. However in stereo vision it is possible under certain circumstances. This is becasue in stereo vision we have only 5 degrees of freedom, 3 perameters for the rotation and 2 for translation. For translation we only know the direction but not the distance due to the scale ambiguity. To show that it is possible to overcome the scale ambiguity we now check how many knows and unknown we have. We have 4 knows for each correspondence. This is bacause we have $u$ and $v$ for both images. Therefore we have $4n$ knowns.
As axplained before w have 5 unknowns for the motion (3 for rotation, 2 for translation) Additionally we have 3 unknowns for each of the correspondences due to the unknown 3D position of their point. To be able to solve such a system we have to have more knowns that unknowns. Therefor the following raltaion must hold: $4n \geq 5+3n$. This hols for $n \geq 5$


## Epopolar Geometry

In this section we want to show that and how it is possible to estimate the relative motion independently fromt he 3D structure.

In Chapter 8 we have introduces the notion of epipolar lines and planes. As a reminder the epipolar plane is the plane which si span by the camera centers as well as the 3D-point. There line in which this planes intercepts the camera plane is the epipolar line. The only positions on the image plane where the image of tha 3D-point can is are along that line.

From the defintion of the epipolar plane we know that the vector from the camera centers to the image of a point on the image plane are coplanar with the vector connecting both camera centers. this leads to the concliustion that the dot-product of the camera-image vector and the normal vector on the epipolar plane is 0. Since we can express the normal vector as the cross product of the line connecting both amera centers (T) and the image etor if the opposing camera we can craate the following formula:

\begin{align*}
p_1 \cdot n = 0 \implies p_1 \cdot (T \times p_2') = 0 \implies p_1 \cdot (T \times Rp_2)
\implies p_1 {[T]}_\times R p_2
\end{align*}

where $p_1'$ is the vator $p_1$ expressed in the coordinates of Camera 2 ($C_2$) and vis versa. We can express ${[T]}_\times R$ ans $E$ whichw e call **essentail matrix**

\begin{align*}
\implies p_1 E p_2 = 0
\end{align*}

This equation is called **Epipolar Constraint** or **Longuet_higgins equation**.

When decomposing E into ${[T]}_\times$ and $R$ has four distinct solutions.

![Epipolar Constraint](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_9/2_epipolar_constraint.png)
*Figure 2: Epipolar Constraint. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


As a short illustration we now look at the case where we have two rectified stereo images. As we have seen in Chapter 4 there is no rotation between rectified images. Therefore we can set $R$ and s 3x3 identity matrix. As a result the essentail matrix only depends on the translation.

![Recified Case](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_9/3_rectified_case.png)
*Figure 3: Recified Case. [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/08_multiple_view_geometry_2.pdf)*


So how many point-corresponedences do we need to be able to calculate the essential matrix. This is a question that many reasearchers have looked into. Kruppa shoed in 1913 that at least 5 points are needed and that in that case there can be up to 11 solutions. In 1988 Demazure showed that actually the are at most 10 distinct solutions. In 2004 Philipp proposed an iterative algorithm for solving this problem. And in 2004 the first non iterative and efficient algorithm was developed by Nister based on the groebner basis decomposistion. However the first popular solution method uses 8 points to get a unique solution and is popular due to its relative easy implementation. This algorithm is called 8-point algorithm or Longuet-Higgins algorithm.

## The 8-point Algorithm